## Confirm rotation and `irreps` of a reaction

In [ ]:
import json

import numpy as np

import plotly.express as px

from ase.visualize import view

from pymatgen.core.structure import Molecule
from pymatgen.io.ase import AseAtomsAdaptor

import basis_set_exchange as bse

import torch

from e3nn import o3

from minimal_basis.transforms.rotations import RotationMatrix
from minimal_basis.predata.matrices import TaskdocsToData

from instance_mongodb import instance_mongodb_sei

In [ ]:
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_sn2_reaction_calculation

basis_info = bse.get_basis("6-31g*", fmt="json", elements=list(range(1, 19)))
basis_info = json.loads(basis_info)

kwargs = {
    "store_extra_tags": "euler_angles"
}

taskdocs_to_data = TaskdocsToData(
    collection=collection,
    filter_collection={'tags.inverted_coordinates': True},
    identifier="idx",
    state_identifier="state",
    reactant_tag="reactant",
    transition_state_tag="transition_state",
    product_tag="product",
    basis_set_type="full",
    basis_info_raw=basis_info,
    d_functions_are_spherical=True,
    **kwargs
)
data = taskdocs_to_data.get_data()

In [ ]:
quantity = "coeff_matrices"
state_idx = 0 # 0 for reactant, 1 for transition state, 2 for product

In [ ]:
all_molecules = [ data[i]['structures'] for i in range(len(data)) ]
all_reactant_atoms = []
all_transition_state_atoms = []
all_product_atoms = []
for _molecules in all_molecules:
    molecules = [Molecule.from_dict(molecule) for molecule in _molecules] 
    all_reactant_atoms += [AseAtomsAdaptor.get_atoms(molecules[0])]
    all_transition_state_atoms += [AseAtomsAdaptor.get_atoms(molecules[1])]
    all_product_atoms += [AseAtomsAdaptor.get_atoms(molecules[2])]
view(all_transition_state_atoms[7], viewer="x3d")

In [ ]:
quantity_to_plot = [ data[i][quantity][state_idx] for i in range(len(data)) ]
quantity_to_plot = np.array(quantity_to_plot)
quantity_to_plot = quantity_to_plot[:, 0, ...]
fig = px.imshow(quantity_to_plot, animation_frame=0, range_color=[-1, 1])
labels = data[0]['orbital_info'][state_idx]
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
eigenvalues = data[0]['eigenvalues'][state_idx][0]
fig.update_xaxes(ticktext=np.round(eigenvalues, 3), tickvals=list(range(len(eigenvalues))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Coefficient matrix from DFT calculation")
# Decrease font size of x and y tick labels 
fig.update_xaxes(tickfont=dict(size=4))
fig.update_yaxes(tickfont=dict(size=4))
fig.show()

In [ ]:
new_coeff_matrix = []
calculated_coeff_matrix = quantity_to_plot 
original_coeff_matrix = calculated_coeff_matrix[0] 

for _idx in range(len(data)):

    angles = data[_idx]['euler_angles'][0]
    torch_angles = torch.tensor(angles, dtype=torch.float64)

    irreps = data[_idx]['irreps'][0]
    irreps = o3.Irreps(irreps)

    rotation_matrix = RotationMatrix(angle_type="euler", angles=angles)() 
    rotation_matrix = torch.tensor(rotation_matrix, dtype=torch.float64)
    
    if _idx == 0:
        rotation_matrix_0 = rotation_matrix
    
    rotation_matrix = rotation_matrix @ rotation_matrix_0.T
     
    D_matrix = irreps.D_from_matrix(rotation_matrix)
    D_matrix = D_matrix.detach().numpy()

    _new_coeff_matrix = np.zeros_like(original_coeff_matrix)
    for i in range(original_coeff_matrix.shape[1]):
        _new_coeff_matrix[:, i] = original_coeff_matrix[:, i] @ D_matrix.T

    new_coeff_matrix.append(_new_coeff_matrix)

new_coeff_matrix = np.array(new_coeff_matrix)

fig = px.imshow(new_coeff_matrix, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
# fig.update_xaxes(ticktext=np.round(data['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Rotated coefficient matrix from D-matrix based on rotated Euler angles")
fig.show()

In [ ]:
difference_matrices = np.abs(new_coeff_matrix) - np.abs(calculated_coeff_matrix)
print(f"Maximum difference: {np.max(difference_matrices)}")
fig = px.imshow(difference_matrices, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
# fig.update_xaxes(ticktext=np.round(data['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Difference between rotated and calculated coefficient matrix")
fig.show()